In [ ]:
import evaluate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from .utils._logger import logger
from .utils._validation import config_args

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from pandas import DataFrame

try:
    df: DataFrame = pd.read_excel(config_args.data_path, index_col=0)
except FileNotFoundError:
    logger.error("File not found.  Ensure it is in the correct location.")
    raise
except Exception as e:
    logger.error(f"Error reading Excel file: {e}")
    raise

In [ ]:
# Data analysis
null_count = df.isnull().sum()
logger.info(f"Null count: {null_count}")

df_copy = df.copy().dropna()

In [ ]:
# Plot class distribution
df_copy["label"].value_counts(ascending=True).plot.bar(color=["green", "blue"])
plt.title("Label Distribution")
plt.show()

In [ ]:
# Calculate average tokens per word
np.mean(df_copy["title"].str.split().apply(len) * 1.5)

df_copy["title_tokens"] = df_copy["title"].str.split().apply(len) * 1.5
df_copy["text_tokens"] = df_copy["text"].str.split().apply(len) * 1.5

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

ax[0].hist(df_copy["title_tokens"], bins=50, color="green")
ax[1].hist(df_copy["text_tokens"], bins=50, color="blue")

In [ ]:
# Data Process
train, test = train_test_split(df_copy, test_size=0.3, stratify=df_copy["label"])
test, validation = train_test_split(test, test_size=1 / 3, stratify=test["label"])

In [ ]:
# Creating dataset
dataset = DatasetDict(
    {
        "train": Dataset.from_pandas(train, preserve_index=False),
        "test": Dataset.from_pandas(test, preserve_index=False),
        "validation": Dataset.from_pandas(validation, preserve_index=False),
    }
)

In [ ]:
accuracy = evaluate.load("accuracy")


def compute_metrics_evaluate(eval_pred):
    """Evaluate metrics"""
    try:
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return accuracy.compute(predictions=predictions, references=labels)
    except Exception as e:
        logger.error(f"Error while evaluate: {e}")


def compute_metrics(pred):
    """
    Return accuracy and F1.
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)

    return {"accuracy": acc, "f1": f1}

In [ ]:
from datasets.dataset_dict import DatasetDict


def train_model(model_ckpt) -> dict[str, float] | None:
    """ "Training the model after tokenizing the batched dataset.
    Args:
        model_ckpt: Model Checkpoint
    """
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
        config = AutoConfig.from_pretrained(
            model_ckpt, label2id={"Real": 0, "Fake": 1}, id2label={0: "Real", 1: "Fake"}
        )
        model = AutoModelForSequenceClassification.from_pretrained(
            model_ckpt, config=config
        ).to(device)
    except Exception as e:
        logger.error(f"Error while initializing model {e}")

    def local_tokenizer(batch):
        """Tokenize the given batch"""
        try:
            temp = tokenizer(batch["title"], padding=True, truncation=True)
            return temp
        except Exception as e:
            logger.error(f"Error while tokenize: {e}")

    encoded_dataset: DatasetDict = dataset.map(
        local_tokenizer, batched=True, batch_size=None
    )

    trainingconfig_args = TrainingArguments(
        output_dir=config_args.output_path,
        overwrite_output_dir=True,
        #   num_train_epochs = 2,
        max_steps=50,
        learning_rate=2e-5,
        per_device_train_batch_size=config_args.batch_size,
        per_device_eval_batch_size=config_args.batch_size,
        weight_decay=0.01,
        eval_strategy="epoch",
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        compute_metrics=compute_metrics,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer,
        args=trainingconfig_args,
    )
    try:
        trainer.train()
    except Exception as e:
        logger.error(f"Error while training {e}")

    preds = trainer.predict(encoded_dataset["test"])

    return preds.metrics

In [ ]:
train_model(config_args.model_checkpoint)